# Actualizacion de la base

Se busca tener actualizada de forma diaria la base para asi poder mantener un analisis enfocado al momento.

In [1]:
# Librerias
import requests
import pandas as pd
from datetime import datetime, timedelta

# Bases y repositorio de los lotes 
ruta_parquet = r"C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\lote_2025_S2.parquet"
chunk_size = 50000
dataset_id = "p6dx-8zbt"
url = f"https://www.datos.gov.co/resource/{dataset_id}.json"


In [5]:
# Último semestre conocido
df_existing = pd.read_parquet(ruta_parquet)
df_existing["fecha_de_publicacion_del"] = pd.to_datetime(df_existing["fecha_de_publicacion_del"], errors="coerce", utc=True)

# --- 2. Definir el rango a actualizar ---
ultima_fecha = df_existing["fecha_de_publicacion_del"].max()
fecha_inicio = (ultima_fecha + timedelta(days=1)).strftime("%Y-%m-%dT00:00:00")
fecha_fin = pd.Timestamp.today().strftime("%Y-%m-%dT23:59:59")

print(f"Actualizando desde: {fecha_inicio} hasta: {fecha_fin}")


Actualizando desde: 2025-09-10T00:00:00 hasta: 2025-09-09T23:59:59


In [6]:
offset = 0
nuevos_dfs = []

while True:
    params = {
        "$limit": chunk_size,
        "$offset": offset,
        "$order": "fecha_de_publicacion_del ASC",
        "$where": f"fecha_de_publicacion_del between '{fecha_inicio}' and '{fecha_fin}'"
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not data:
        break
    df_chunk = pd.DataFrame(data)
    df_chunk["fecha_de_publicacion_del"] = pd.to_datetime(df_chunk["fecha_de_publicacion_del"], errors="coerce", utc=True)
    nuevos_dfs.append(df_chunk)
    offset += chunk_size
    print(f"  ✅ Traídos {len(df_chunk):,} registros (offset {offset})")

# --- 4. Concatenar con los existentes ---
if nuevos_dfs:
    df_new = pd.concat(nuevos_dfs, ignore_index=True)
    df_actualizado = pd.concat([df_existing, df_new], ignore_index=True)
    df_actualizado.to_parquet(ruta_parquet, engine="pyarrow", index=False)
    print(f"\n✅ Actualización completada. Total registros: {len(df_actualizado):,}")
else:
    df_actualizado = df_existing
    print("\n✅ No hay registros nuevos para actualizar.")

# --- 5. Resumen de los últimos días ---
df_actualizado['fecha'] = df_actualizado['fecha_de_publicacion_del'].dt.date
ultimos_dos_dias = df_actualizado['fecha'].value_counts().sort_index(ascending=False).head(2)
print("\n📊 Últimos días y cantidad de registros:")
print(ultimos_dos_dias)




✅ No hay registros nuevos para actualizar.

📊 Últimos días y cantidad de registros:
fecha
2025-09-09     3
2025-09-08    21
Name: count, dtype: int64


In [7]:
import shutil
origen = "Actualizacion de Datos SECOP.ipynb"
destino = r"C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\Actualizacion de Datos SECOP.ipynb"

shutil.copy(origen, destino)
print(f"✅ Notebook copiado a: {destino}")


✅ Notebook copiado a: C:\Users\aleja\OneDrive\Escritorio\Consultoria\desarrollo_social_y_economico\Lotes\Actualizacion de Datos SECOP.ipynb
